In [23]:
from pulp import *
import pandas as pd

Solution adapted from the 

Python PuLP solution prepared by Thomas W. Miller for MSDS 460
Revised April 20, 2023
Implemented using activities dictionary with derived start_times and end_times
rather than time decision variables as in Williams (2013)

In [33]:
# Adjust the activities to match software development phases
activities_best = {
    'A': 8,  
    'B': 16,     
    'C': 8,          
    'D1': 16,             
    'D2': 40,          
    'D3': 40,    
    'D4': 80,      
    'D5': 16,         
    'D6': 24, 
    'D7': 24,   
    'D8': 8,        
    'E': 19,          
    'F': 8,          
    'G': 24,      
    'H':16,       
}


activities_expected = {
    'A': 11,  # Based on expectedHours from the table
    'B': 21,
    'C': 12,
    'D1': 19,
    'D2': 60,
    'D3': 60,
    'D4': 100,
    'D5': 32,
    'D6': 41,
    'D7': 41,
    'D8': 16,
    'E': 33,
    'F': 16,
    'G': 27,
    'H': 19
}

activities_worst = {
    'A': 24,  # Based on worstCaseHours from the table
    'B': 32,
    'C': 24,
    'D1': 32,
    'D2': 80,
    'D3': 80,
    'D4': 120,
    'D5': 48,
    'D6': 60,
    'D7': 60,
    'D8': 24,
    'E': 48,
    'F': 24,
    'G': 40,
    'H': 32
}


# Map of tasks and the number of people working on each task
people_per_task = {
    'A': 1,  
    'B': 1,     
    'C': 1,          
    'D1': 3,  # note the adjustment on 4 to three people - this is intentional to account for the amount of collaboration time required as more people work in a small space        
    'D2': 3,          
    'D3': 3,    
    'D4': 3,      
    'D5': 3,         
    'D6': 3, 
    'D7': 3,   
    'D8': 3,        
    'E': 1,          
    'F': 1,  # this is three people involved, but two in a helping capacity only        
    'G': 4,   # this is the whole team, but adjusting down by 1
    'H': 1,       
}

 # Adjust precedences to reflect a software development process
precedences = {
    'A': [],  
    'B': [],     
    'C': ['A'],          
    'D1': ['A'],             
    'D2': ['D1'],          
    'D3': ['D1'],    
    'D4': ['D2', 'D3'],      
    'D5': ['D4'],         
    'D6': ['D4'], 
    'D7': ['D6'],   
    'D8': ['D5', 'D7'],        
    'E': ['B', 'C'],          
    'F': ['D8', 'E'],          
    'G': ['A', 'D8'],      
    'H': ['F', 'G'], 
}

activity_set = {
    'best' : activities_best,
    'expected' : activities_expected,
    'worst' : activities_worst
}


In [32]:
# Function to adjust the hours based on the number of people
def convert_days_by_people(activities, people_per_task, hours_per_day=6):
    adjusted_days = {}
    for task, hours in activities.items():
        people = people_per_task.get(task, 1)  # Get number of people or default to 1 if not provided
        adjusted_hours = hours / people  # Divide hours by the number of people
        adjusted_days[task] = round(adjusted_hours / hours_per_day, 2)  # Convert to days
    return adjusted_days

In [34]:
for activity_name, activity_map in activity_set.items():

    # Get the adjusted days for each task
    adjusted_activity_days = convert_days_by_people(activity_map, people_per_task)

    # Create a list of the activities
    activities_list = list(adjusted_activity_days.keys())

    # Create the LP problem
    prob = LpProblem("Critical Path", LpMinimize)

    # Create the LP variables for start and end times
    start_times = {activity: LpVariable(f"start_{activity}", 0, None) for activity in activities_list}
    end_times = {activity: LpVariable(f"end_{activity}", 0, None) for activity in activities_list}

    # Add constraints to the LP problem
    for activity in activities_list:
        # Activity duration constraint
        prob += end_times[activity] == start_times[activity] + adjusted_activity_days[activity], f"{activity}_duration"
        
        # Precedence constraints (each task must start after its predecessors end)
        for predecessor in precedences[activity]:
            prob += start_times[activity] >= end_times[predecessor], f"{activity}_predecessor_{predecessor}"

    # Set the objective function to minimize the total project time
    prob += lpSum([end_times[activity] for activity in activities_list]), "minimize_end_times"

    # Solve the LP problem
    status = prob.solve()

    # Print the critical path results
    print(f"Critical Path time for {activity_name}:")
    for activity in activities_list:
        print(f"{activity} starts at {value(start_times[activity])} and ends at {value(end_times[activity])}")

    # Print solution variable values
    print("\nSolution variable values:")
    for var in prob.variables():
        if var.name != "_dummy":
            print(var.name, "=", var.varValue)

    task_data = []
    for activity in activities_list:
        start_value = value(start_times[activity]) + 1  # Extract the start time value so the gantt chart looks right
        end_value = value(end_times[activity]) + 1     # Extract the end time value so the gantt chart looks right
        task_data.append({
            'Activity': activity,
            'Start Time': start_value,
            'End Time': end_value
        })


    df = pd.DataFrame(task_data)

    # Save the DataFrame to a CSV file
    csv_filename = f'/Users/gracefujinaga/Documents/Northwestern/MSDS 460/project_planning/{activity_name}_task_times.csv'
    df.to_csv(csv_filename, index=False)



/opt/homebrew/lib/python3.9/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/zv/4f9cw9vs6tjbvz5bh2k07w_40000gn/T/7b7e816253ef4d828000f57159d68d63-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/zv/4f9cw9vs6tjbvz5bh2k07w_40000gn/T/7b7e816253ef4d828000f57159d68d63-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 39 COLUMNS
At line 123 RHS
At line 158 BOUNDS
At line 159 ENDATA
Problem MODEL has 34 rows, 30 columns and 68 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-34) rows, 0 (-30) columns and 0 (-68) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 118.25
After Postsolve, objective 118.25, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 118.25 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed fr